In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import pandas as pd
import numpy as np

# --- Configuração do Driver
# --- Caminho do driver e fonte dos dados que serão recolhidos
CHROME_DRIVER_PATH = 'drivers/chromedriver.exe'
MAL_SEASON_URL = "https://myanimelist.net/anime/season"

# Listas para armazenar os dados coletados
titulos = []
scores = []
membros = []
generos = []
episodios = []
estudios = []

print("Iniciando o Web Scraper do MyAnimeList...")

try:
    service = Service(executable_path=CHROME_DRIVER_PATH)
    driver = webdriver.Chrome(service=service)
    print("Navegador Chrome iniciado.")
    driver.implicitly_wait(10)
    driver.get(MAL_SEASON_URL)
    print(f"Acessando a URL: {MAL_SEASON_URL}")
    time.sleep(5)

    anime_containers = driver.find_elements(By.CLASS_NAME, 'seasonal-anime')
    print(f"Encontrados {len(anime_containers)} animes na página.")

    for anime in anime_containers:
        title_text = 'N/A'
        score_text = 'N/A'
        members_text = 'N/A'
        genres_list = []
        episodes_text = 'N/A' # Reset para cada anime
        studio_text = 'N/A'   # Reset para cada anime

        try:
            # --- Coleta de Título, Score, Membros, Gêneros (permanecem como estavam) ---
            title_element = anime.find_element(By.CLASS_NAME, 'h2_anime_title')
            title_text = title_element.text.strip()

            try:
                score_element = anime.find_element(By.CSS_SELECTOR, '.scormem-item.score .score-label')
                score_text = score_element.text.strip()
            except NoSuchElementException:
                try:
                    score_element = anime.find_element(By.CLASS_NAME, 'score')
                    score_text = score_element.text.strip()
                except NoSuchElementException:
                    pass

            try:
                members_element = anime.find_element(By.CLASS_NAME, 'scormem-item.member')
                members_text = members_element.text.replace('members', '').strip()
            except NoSuchElementException:
                pass

            try:
                genres_container = anime.find_element(By.CLASS_NAME, 'genres.js-genre')
                genre_elements = genres_container.find_elements(By.TAG_NAME, 'a')
                genres_list = [g.text.strip() for g in genre_elements if g.text.strip()]
            except NoSuchElementException:
                pass
            
            try:
                # Encontra o contêiner 'info'
                info_div = anime.find_element(By.CLASS_NAME, 'info')
                # Dentro do 'info', encontra TODOS os spans com a classe 'item'
                item_spans = info_div.find_elements(By.CLASS_NAME, 'item')
                for item_span in item_spans:
                    # Verifica se o texto do span ou de seus filhos contém 'eps'
                    if "eps" in item_span.text:
                        episodes_text = item_span.text.strip()
                        # Se houver parênteses com minutos (ex: "12 eps (24 min)"), limpa
                        if "(" in episodes_text and ")" in episodes_text:
                            episodes_text = episodes_text.split('(')[0].strip()
                        break # Encontrou, pode sair do loop de itens
            except NoSuchElementException:
                pass # Permanece 'N/A'
            
            try:
                # Encontra o contêiner 'properties'
                properties_div = anime.find_element(By.CLASS_NAME, 'properties')
                studio_item_element = properties_div.find_element(By.XPATH, ".//div[@class='property'][./span[@class='caption' and text()='Studio']]//span[@class='item']")
                studio_text = studio_item_element.text.strip()
                
            except NoSuchElementException:
                pass # Permanece 'N/A'

        except NoSuchElementException as e:
            print(f"Elemento não encontrado para o anime '{title_text if title_text != 'N/A' else 'Desconhecido'}': {e}")
        except Exception as e:
            print(f"Ocorreu um erro inesperado ao processar o anime '{title_text if title_text != 'N/A' else 'Desconhecido'}': {e}")
        
        titulos.append(title_text)
        scores.append(score_text)
        membros.append(members_text)
        generos.append(", ".join(genres_list))
        episodios.append(episodes_text)
        estudios.append(studio_text)

        print(f"Coletado: Título: '{title_text}', Score: '{score_text}', Membros: '{members_text}', Gêneros: '{', '.join(genres_list)}', Episódios: '{episodes_text}', Estúdio: '{studio_text}'")

except TimeoutException:
    print("Tempo limite excedido. A página pode não ter carregado a tempo.")
except Exception as e:
    print(f"Ocorreu um erro geral ao iniciar ou processar o scraping: {e}")

finally:
    if 'driver' in locals() and driver:
        driver.quit()
        print("Navegador Chrome fechado.")

print("\n--- Dados Coletados Brutos ---")
print(f"Total de animes coletados: {len(titulos)}")


Iniciando o Web Scraper do MyAnimeList...
Navegador Chrome iniciado.
Acessando a URL: https://myanimelist.net/anime/season
Encontrados 197 animes na página.
Coletado: Título: 'Dandadan 2nd Season', Score: '8.47', Membros: '291K', Gêneros: 'Action, Comedy, Supernatural', Episódios: '12 eps, 24 min', Estúdio: 'Science SARU'
Coletado: Título: 'Sono Bisque Doll wa Koi wo Suru Season 2', Score: '8.44', Membros: '285K', Gêneros: 'Romance', Episódios: '12 eps, 23 min', Estúdio: 'CloverWorks'
Coletado: Título: 'Kaijuu 8-gou 2nd Season', Score: '7.94', Membros: '189K', Gêneros: 'Action, Fantasy, Sci-Fi', Episódios: '11 eps, 23 min', Estúdio: 'Production I.G'
Coletado: Título: 'Kaoru Hana wa Rin to Saku', Score: '8.80', Membros: '185K', Gêneros: 'Drama, Romance', Episódios: '13 eps, 24 min', Estúdio: 'CloverWorks'
Coletado: Título: 'Gachiakuta', Score: '8.09', Membros: '174K', Gêneros: 'Action, Fantasy', Episódios: '24 eps, 23 min', Estúdio: 'Bones Film'
Coletado: Título: 'Yofukashi no Uta Seaso

In [3]:
# --- NOVA ETAPA: SALVAR DADOS BRUTOS EM CSV ---
print("\nSalvando dados brutos em 'animes_mal_raw.csv'...")

try:
    # 1. Criar um DataFrame Pandas temporário com os dados brutos
    df_animes_raw = pd.DataFrame({
        'Titulo': titulos,
        'Score': scores,
        'Membros': membros,
        'Generos': generos,
        'Episodios': episodios,
        'Estudios': estudios
    })

    # 2. Salvar o DataFrame em um arquivo CSV
    # index=False evita que o Pandas escreva o índice do DataFrame como uma coluna no CSV
    # encoding='utf-8' garante que caracteres especiais (como nomes de animes) sejam salvos corretamente
    df_animes_raw.to_csv('animes_mal_raw.csv', index=False, encoding='utf-8')
    print("Dados brutos salvos com sucesso em 'animes_mal_raw.csv'.")

except Exception as e:
    print(f"Erro ao salvar dados brutos em CSV: {e}")


Salvando dados brutos em 'animes_mal_raw.csv'...
Dados brutos salvos com sucesso em 'animes_mal_raw.csv'.
